## 決定木デフォルト

In [1]:
import numpy as np
import pandas as pd

from sklearn import tree
from sklearn.metrics import  mean_squared_error
from sklearn.model_selection import train_test_split

import graphviz

In [2]:
# pickle ファイルから読み込み
train_pkl = pd.read_pickle('./train.pk3')

In [3]:
train_pkl.shape

(55323, 15)

In [4]:
train_pkl.describe()

,id,accommodates,bathrooms,bedrooms,beds,host_response_rate,number_of_reviews,y,cancellation_policy_moderate,cancellation_policy_strict,cancellation_policy_super_strict_30,cancellation_policy_super_strict_60,room_type_Private room,room_type_Shared room,property_type_num
count,55323.000000,55323.000000,55323.000000,55323.000000,55323.000000,5.532300e+04,55323.000000,55323.000000,55323.000000,55323.000000,55323.000000,55323.000000,55323.000000,55323.000000,55323.000000
mean,27787.181588,0.087360,0.248414,0.111274,0.238273,2.311836e-17,0.204165,160.307341,0.257253,0.437467,0.001356,0.000235,0.413806,0.028975,0.999259
std,16043.404061,0.790027,0.782466,0.794160,0.712608,1.000009e+00,0.752675,168.266655,0.437124,0.496079,0.036795,0.015328,0.492519,0.167739,0.075455
min,0.000000,-1.000000,-2.000000,-1.000000,-1.000000,-1.651052e+00,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,13892.500000,0.000000,0.000000,0.000000,0.000000,-1.651052e+00,0.000000,74.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,27778.000000,0.000000,0.000000,0.000000,0.000000,6.706177e-01,0.000000,111.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,41681.500000,0.000000,0.000000,0.000000,0.000000,6.706177e-01,0.000000,185.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000
max,55582.000000,5.000000,11.000000,10.000000,12.000000,6.706177e-01,15.000000,1999.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000


In [5]:
train_pkl.head()

,id,accommodates,bathrooms,bedrooms,beds,host_response_rate,number_of_reviews,y,cancellation_policy_moderate,cancellation_policy_strict,cancellation_policy_super_strict_30,cancellation_policy_super_strict_60,room_type_Private room,room_type_Shared room,property_type_num
0,0,1,1,0,1,-1.651052,0,138.0,0,0,0,0,1,0,1
1,1,0,0,0,0,0.670618,0,42.0,0,1,0,0,1,0,1
2,2,0,1,0,0,0.670618,0,65.0,0,1,0,0,1,0,1
3,3,0,0,0,0,0.670618,0,166.0,0,1,0,0,1,0,1
4,4,0,0,0,0,0.670618,0,165.0,0,1,0,0,0,0,1


## 訓練データとテストデータに分割

In [6]:
# 訓練とテストデータに分割
train, test = train_test_split(train_pkl, test_size=0.2, random_state=42)

In [7]:
# ターゲットと特徴量の分割
train_X_tmp = train.copy()
train_X_tmp.drop(['id', 'y'], axis=1, inplace=True)
train_X = train_X_tmp.iloc[:].values
train_y = train.y.values

In [8]:
train_X.shape, train_y.shape, train_X_tmp.shape

((44258, 13), (44258,), (44258, 13))

## 訓練データで訓練

In [9]:
DT = tree.DecisionTreeRegressor(max_depth=5, random_state=42)

In [10]:
DT = DT.fit(train_X, train_y)

In [11]:
train.y.name

'y'

In [12]:
DT.feature_importances_

array([0.02704558, 0.24604367, 0.49216345, 0.00394997, 0.02245871,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.16315611, 0.04351772, 0.00166479])

In [13]:
# train_X.columns

In [14]:
# 特徴量の重要度が高い順に表示
print("特徴量の重要度が高い順：")
# sorted：reverse=True 降順
print(sorted(
    zip(map(lambda x: round(x, 3), DT.feature_importances_), train.iloc[:, [2, 3, 4, 5, 7, 8, 9, 10, 11, 12]].columns),
    reverse=True))

特徴量の重要度が高い順：
[(0.492, 'beds'), (0.246, 'bedrooms'), (0.027, 'bathrooms'), (0.022, 'y'), (0.004, 'host_response_rate'), (0.0, 'room_type_Private room'), (0.0, 'cancellation_policy_super_strict_60'), (0.0, 'cancellation_policy_super_strict_30'), (0.0, 'cancellation_policy_strict'), (0.0, 'cancellation_policy_moderate')]


In [15]:
# len(train.iloc[:, 1:].columns), train.iloc[:, 1:].columns

In [16]:
# 訓練済みの決定木を視覚化
# dot_data = tree.export_graphviz(DT, out_file=None,
#                                feature_names=train.iloc[:, 1:-1].columns,
#                                class_names=train.y.name,
#                                rounded=True,
#                                filled=True,
#                                special_characters=True)

In [17]:
# graph = graphviz.Source(dot_data)
# graph

In [18]:
# graph.write('.\DT.png')

In [19]:
type(train_X)

numpy.ndarray

In [24]:
sorted(
    zip(map(lambda x: round(x, 3), DT.feature_importances_), train.iloc[:, [2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14]].columns),
    reverse=True)

[(0.492, 'beds'),
 (0.246, 'bedrooms'),
 (0.163, 'room_type_Shared room'),
 (0.044, 'property_type_num'),
 (0.027, 'bathrooms'),
 (0.022, 'number_of_reviews'),
 (0.004, 'host_response_rate'),
 (0.0, 'room_type_Private room'),
 (0.0, 'cancellation_policy_super_strict_60'),
 (0.0, 'cancellation_policy_super_strict_30'),
 (0.0, 'cancellation_policy_strict'),
 (0.0, 'cancellation_policy_moderate')]

## テストデータで実行

In [25]:
# ターゲットと特徴量の分割
test_X_tmp = test.copy()
test_X_tmp.drop(['id', 'y'], axis=1, inplace=True)
test_x = test_X_tmp.iloc[:].values
test_y = test.y.values

In [26]:
test_x.shape, test_y.shape

((11065, 13), (11065,))

In [27]:
pred_y = DT.predict(test_x)

In [28]:
pred_y.shape

(11065,)

In [29]:
np.sqrt(mean_squared_error(test_y, pred_y))

133.36444010894084

In [30]:
test.head()

,id,accommodates,bathrooms,bedrooms,beds,host_response_rate,number_of_reviews,y,cancellation_policy_moderate,cancellation_policy_strict,cancellation_policy_super_strict_30,cancellation_policy_super_strict_60,room_type_Private room,room_type_Shared room,property_type_num
43933,43933,0,0,0,0,0.670618,0,75.0,0,1,0,0,1,0,1
277,277,0,0,0,0,0.670618,0,79.0,0,1,0,0,0,0,1
1799,1799,0,0,0,0,0.670618,2,41.0,0,1,0,0,1,0,1
49693,49693,0,0,-1,0,0.438451,0,145.0,0,1,0,0,0,0,1
14870,14870,1,1,0,2,0.670618,0,175.0,0,0,0,0,0,0,1


## 検証データで実行

In [31]:
# 検証データ読み込み
valid = pd.read_pickle('./test.pk3')

In [32]:
valid.shape

(18528, 14)

In [33]:
# ID の保存
valid_pass = valid.id.values

In [34]:
valid_X = valid.iloc[:, 1:].values

In [35]:
# valid_X.describe()

In [36]:
valid_X.shape, train_X.shape

((18528, 13), (44258, 13))

In [37]:
pred_valid_y = DT.predict(valid_X)

In [38]:
pred_valid_y.shape

(18528,)

In [39]:
type(valid_pass), type(pred_valid_y)

(numpy.ndarray, numpy.ndarray)

In [40]:
result_df = pd.DataFrame(pred_valid_y, valid_pass, columns=['y'])

In [41]:
result_df.head()

,y
0,275.434567
1,168.539394
2,85.867222
3,168.539394
4,168.539394


In [42]:
result_df.to_csv("./tree_8.csv", header=False)